Выкачиваем отзывы на ноутбуки. Для этого нам понадобится:

In [ ]:
!pip3 install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
from tqdm.auto import tqdm
import random
import re
from bs4 import BeautifulSoup
import requests

Парсим:

In [ ]:
def parse_site(url: str, ua: str):
    '''
    Parsing of site (html) using BeautifulSoup.
    Returns bs4 object.
    '''

    res = requests.get(url, headers={'User-Agent': ua})
    page = res.text
    soup = BeautifulSoup(page, 'html.parser')

    return soup

In [ ]:
USER_AGENTS = ['Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/89.0',
'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:91.0) Gecko/20100101 Firefox/91.0',
'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.43 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36 OPR/77.0.4054.277',
'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
'Mozilla/5.0 (Windows NT 6.3) AppleWebKit/537.43 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36 OPR/84.0.4316.31']

In [ ]:
def parse_prod_urls(ctlg_url: str, ua: str) -> list:
    '''
    Function that parse references on the products.
    '''

    soup_list_prod = parse_site(ctlg_url, ua)
    list_main_sent = soup_list_prod.find('div', {'class': 'main-sentences-list'})
    picture_blocks = list_main_sent.find_all('div', {'class': 'left-item-block'})
    list_prods_hrefs = [block.find('a').attrs['href'] for block in picture_blocks]

    return list_prods_hrefs

Without stars:

In [ ]:
def parse_revs_for_prod(list_prods: list, list_uas: list) -> list:
    '''
    Function that parse reviews from references.
    '''

    list_reviews = []

    for prod in list_prods:

        match = re.search('/(.+?)/$', prod)
        product = match.group(0)  # имя продукта

        revs = [product]

        time.sleep(random.randint(3, 6))

        prod_url = 'https://otzivi-tut.ru' + prod
        soup_prod = parse_site(prod_url, random.choice(list_uas))
        rev_blocks = soup_prod.find_all('div', {'class': 'rew-item'})
        # print(prod_url)

        if rev_blocks:

            for rev_block in rev_blocks:

                list_rev = []

                # stars_block = rev_block.find('span', {'class': 'stars'})
                # list_full_stars = stars_block.find_all('span', {'class': 'star full'})
                # count_stars = len(list_full_stars)

                rev_text_block = rev_block.find('div', {'class': 'text-s'})
                rev_text = rev_text_block.text

                # list_rev.append(count_stars)
                # list_rev.append(rev_text)

                revs.append(rev_text)

        list_reviews.append(revs)

    return list_reviews

With stars:

In [ ]:
def parse_reviews(list_prods: list, list_uas: list) -> list:
    '''
    Функция, собирающая отзывы на продукты по ссылкам с сайта.
    Возвращаем список отзывов на продукты.
    '''

    # список отзывов без разделения на продукты
    list_reviews = []

    for prod in list_prods:

        revs = []

        time.sleep(random.randint(3, 8))  # спим, чтобы не забанили за частые запросы

        prod_url = 'https://otzivi-tut.ru' + prod
        soup_prod = parse_site(prod_url, random.choice(list_uas))
        rev_blocks = soup_prod.find_all('div', {'class': 'rew-item'})
        # print(prod_url)

        if rev_blocks:

            for rev_block in rev_blocks:

                try:

                    # считаем количество звёздочек
                    # которые поставил пользователь на продукт
                    stars_block = rev_block.find('span', {'class': 'stars'})
                    list_full_stars = stars_block.find_all('span', {'class': 'star full'})
                    count_stars = len(list_full_stars)

                    # отзыв
                    rev_text_block = rev_block.find('div', {'class': 'text-s'})
                    rev_text = rev_text_block.text

                    if rev_text:  # бывает, парсятся пустыми
                        # print('Не пусто')
                        match_1 = re.search('Достоинства', rev_text)
                        match_2 = re.search('Плюсы', rev_text)

                        if match_1 or match_2:

                            final_rev_text = rev_text + '\n' + f'Количество звёзд на отзыве: {count_stars}'
                            revs.append(final_rev_text)

                except:
                    print('Что-то пошло не так с парсингом отзыва!', prod)

        if len(revs) >= 10:

            list_reviews += revs[:10]

    return list_reviews

One product:

In [ ]:
def preprocess_review(review: str) -> str:
    '''
    Предобработка текста отзыва перед записью в корпус.
    '''
    review = re.sub('\s+', ' ', review)
    review = re.sub('^\s', '', review)
    review = re.sub('\s$', '', review)
    review = re.sub('<br>', '', review)
    review = re.sub('Минусы', ' Минусы', review)
    review = re.sub('Отзыв', ' Отзыв', review)

    return review

In [ ]:
def parse_reviews_for_product(href: str, ua: str) -> list:
    '''
    Функция, которая собирает 10 отзывов для определённого продукта,
    которые находятся по ссылке.
    '''
    
    revs = []

    time.sleep(random.randint(3, 8))  # спим, чтобы не забанили за частые запросы

    prod_url = 'https://otzivi-tut.ru' + href
    soup_prod = parse_site(prod_url, ua)
    rev_blocks = soup_prod.find_all('div', {'class': 'rew-item'})
    # print(prod_url)

    if rev_blocks:
        for rev_block in rev_blocks:

            try:
                # считаем количество звёздочек
                # которые поставил пользователь на продукт
                stars_block = rev_block.find('span', {'class': 'stars'})
                list_full_stars = stars_block.find_all('span', {'class': 'star full '})
                count_stars = len(list_full_stars)

                # отзыв
                rev_text_block = rev_block.find('div', {'class': 'text-s'})
                rev_text = rev_text_block.text

                if rev_text:  # бывает, парсятся пустыми
                    # print('Не пусто')

                    # проверяем, что есть хотя бы плюсы
                    match_1 = re.search('Достоинства:', rev_text)
                    match_2 = re.search('Плюсы:', rev_text)

                    if match_1 or match_2:

                        final_rev_text = rev_text + ' ' + f'Количество звёзд на отзыве: {count_stars}'
                        final_rev_text = preprocess_review(final_rev_text)
                        revs.append(final_rev_text)

            except:
                print('Что-то пошло не так с парсингом отзыва на', href)

    if len(revs) >= 10:  # если десять или больше выбираем 10
        chosen = revs[:10]
        return chosen

In [ ]:
# list_hrefs = []

# for i in range(2, 7):
time.sleep(random.randint(1, 5))
asc = f'https://otzivi-tut.ru/category/kompyutery/noutbuki/?sort=cnt_asc&PAGEN_1=3'
# print('Новая страница', asc)
list_asc_hrefs = parse_prod_urls(asc, random.choice(USER_AGENTS))
# list_hrefs.extend(list_asc_hrefs)
print(list_asc_hrefs)

['/products/10401/noutbuki/apple-macbook-air-13-mid-2012/', '/products/115971/noutbuki/lenovo-g580-core-i3-3120m-2500-mhz1561366x76840gb320gbdvd-rwnvidia-geforce-gt-610mwi-fibluetoothdos/', '/products/108322/noutbuki/lenovo-g580-pentium-b960-2200-mhz1561366x76840gb1000gbdvd-rwnvidia-geforce-gt-635mwi-fidos/', '/products/101505/noutbuki/lenovo-g580-core-i5-3230m-2600-mhz1561366x76840gb1000gbdvd-rwnvidia-geforce-gt-610mwi-fibluetoothwin-8-64/', '/products/98493/noutbuki/lenovo-g580-pentium-2020m-2400-mhz1561366x76820gb320gbdvd-rwnvidia-geforce-710mwi-fibluetoothwin-8-64/', '/products/103423/noutbuki/lenovo-g580-pentium-b960-2200-mhz1561366x76840gb500gbdvd-rwwi-fiwin-8-64/', '/products/100437/noutbuki/lenovo-g580-celeron-1000m-1800-mhz1561366x76830gb500gbdvd-rwintel-hd-graphics-2000wi-fibluetoothdos/', '/products/113293/noutbuki/lenovo-g580-core-i3-3110m-2400-mhz1561366x76840gb500gbdvd-rwnvidia-geforce-gt-635mwi-fidos/', '/products/11158/noutbuki/acer-aspire-5-a517-51g/', '/products/11258

In [ ]:
parse_reviews_for_product(list_asc_hrefs[0], random.choice(USER_AGENTS))

['Плюсы: – Легкий – Мощный – Вместительный – Стабильный – Нет DVD привода – SSD винт - грузится моментально – Долго держит зарядку Минусы: – Матрица – Клавиатура отпечатывается на экране – Нет HDMI – Микрофон сбоку Отзыв: Ноутбук корпоративный, я за него денег не платил, поэтому не сильно требователен в оценке. Качество и удобство ОС полностью устраивает. Но для Apple качество сборки оставляет желать лучшего. А именно... Во-первых на моем макбуке и на аналогичных макбуках коллег на экране отпечатывается клавиатура. Со временем она так сильно отпечаталась, что уже ничем не стирается. Во-вторых под разными углами (в том числе даже если смотреть прямо) на экране видны более яркие пятна разного размера. Под определенным углом в ценре проходит "млечный путь". Возможно это брак именно моей модели. Там где расположены магнитики постоянно скапливается пыль. На практически идеальном корпусе постоянно видны несколько прямоугольников этой самой пыли. Часто приходится проводить онлайн конференции,

References:

In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/final_pc_hrefs', 'r', encoding='utf-8') as ff:
    hr = ff.read().split('\n')

In [ ]:
print(len(hr))

1480


In [ ]:
hrs = []
short_names = []

for h in hr:
    # print('Имя', name)
    h_name = h.split('/')[-2]
    name_words = []
    ind = True
    for word in h_name.split('-'):
        if ind:
            # print('Слово в имени', word)
            name_words.append(word)
            if re.search('\d+', word) and re.search('\w+', word):
                # print('Нашли цифры и буквы', word)
                ind = False
    # print(name_words)
    short_name = ' '.join(name_words)
    if short_names not in short_names:
        short_names.append(short_name)
        hrs.append(h)

print(short_names[:5])
print(hrs[:5])
print('\n')
print('Итого продуктов в корпусе', len(hrs))

['apple macbook pro 15', 'asus zenbook ux52vs', 'hp pavilion dv6', 'acer aspire one aod270', 'asus k52f']
['/products/117174/noutbuki/apple-macbook-pro-15-with-retina-display-mid-2015-mjlq2-core-i7-2200-mhz1542880x1800160gb256gbdvd-netintel-iris-pro-graphics-5200wi-fibluetoothmacos-x/', '/products/117442/noutbuki/asus-zenbook-ux52vs-core-i7-3517u-1900-mhz1561920x10804096mb500gbdvd-rwwi-fibluetoothwin-8-64/', '/products/115892/noutbuki/hp-pavilion-dv6-2045el-core-i7-720qm-1600-mhz1561366x7684096mb320gbdvd-rwwi-fibluetoothwin-7-hp/', '/products/117935/noutbuki/acer-aspire-one-aod270-268kk/', '/products/123502/noutbuki/asus-k52f-pentium-p6100-2000-mhz1561366x7682048mb3200gbdvd-rwwi-fidos/']


Итого продуктов в корпусе 1480


**Unique products with names and references:**

In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/laptop_hrefs_before_parsing.txt', 'w', encoding='utf-8') as bp:
    bp.write('\n'.join(hrs))

In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/laptop_names_before_parsing.txt', 'w', encoding='utf-8') as nbp:
    nbp.write('\n'.join(short_names))

In several stages:

In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/laptop_hrefs_before_parsing.txt', 'r', encoding='utf-8') as h_f:
    hrs = h_f.read().split('\n')

In [ ]:
hrs_3 = hrs[:3]
print(hrs_3)

['/products/117174/noutbuki/apple-macbook-pro-15-with-retina-display-mid-2015-mjlq2-core-i7-2200-mhz1542880x1800160gb256gbdvd-netintel-iris-pro-graphics-5200wi-fibluetoothmacos-x/', '/products/117442/noutbuki/asus-zenbook-ux52vs-core-i7-3517u-1900-mhz1561920x10804096mb500gbdvd-rwwi-fibluetoothwin-8-64/', '/products/115892/noutbuki/hp-pavilion-dv6-2045el-core-i7-720qm-1600-mhz1561366x7684096mb320gbdvd-rwwi-fibluetoothwin-7-hp/']


Sample:

In [ ]:
parse_3 = parse_reviews(hrs[:3], USER_AGENTS)
print('Длина списка отзывов:', len(parse_3))
print('Три первых отзыва:', parse_3[:3])

Длина списка отзывов: 30
Три первых отзыва: ['\n\nПлюсы: Unix\r\nТрекпад\r\nКачественный экран\r\nКачество сборки\r\nДолгое время работы от аккумулятора\r\nОтличный звук от колонокМинусы: Трекпад начал пощелкивать спустя 3 месяца после покупки\r\nАллюминиевый корпус вминается, если уронить/ударить ноутбук Отзыв: Я работаю разработчиком и покупал ноутбук именно для работы.\r\nПользуюсь ноутбуком уже 5-й месяц. \r\nДо этого пару лет пользовался Thinkpad x220 и Linux. Очень радует то, что операционная система семейства Unix, а также то, что в интернете огромное количество информации под macOS и OS X для разработчиков, когда нужно что-то скомпилировать/собрать и т.д.\r\nЭто как Linux, только очень качественный и доведенный до ума без его болячек, вроде того, что что-нибудь отвалится в процессе работы само по себе и т.д.\r\nИз недостатков:\r\n- корпус можно было бы сделать из более прочного материала - я один раз уронил ноутбук на кафель и получил неплохую вмятину на крышке.\r\n- не знаю с 

Pipeline:
* Get one link
* Get reviews from the link

In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/laptop_names_before_parsing.txt', 'r', encoding='utf-8') as sf:
    short_names = sf.read().split('\n')
with open('/content/drive/MyDrive/Summarization/laptop data/laptop_hrefs_before_parsing.txt', 'r', encoding='utf-8') as pf:
    pc_hrefs = pf.read().split('\n')

In [ ]:
for name, pc_href in zip(short_names, pc_hrefs):  # zip по двум итераторам
    print(name, pc_href)
    break

apple macbook pro 15 /products/117174/noutbuki/apple-macbook-pro-15-with-retina-display-mid-2015-mjlq2-core-i7-2200-mhz1542880x1800160gb256gbdvd-netintel-iris-pro-graphics-5200wi-fibluetoothmacos-x/


Sample:

In [ ]:
for idx, (name, href) in enumerate(zip(short_names, pc_hrefs)):
    print(idx)
    print(name)
    print(href)

    time.sleep(random.randint(1, 5))

    try:
        product_reviews = parse_reviews_for_product(href, random.choice(USER_AGENTS))
        # processing
        if product_reviews:  # if there are less than 10 reviews on the product, nothing is returned

            # reviews
            with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_reviews.txt', 'a', encoding='utf-8') as cf:
                # запись
                cf.write('\n'.join(product_reviews))
                cf.write('\n')

            # name
            with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_names.txt', 'a', encoding='utf-8') as cnf:
                cnf.write(name)
                cnf.write('\n')

            # link
            with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_hrefs.txt', 'a', encoding='utf-8') as chf:
                chf.write(href)
                chf.write('\n')
    except:
        print('There is an error!')

    if idx % 100 == 0:
        print(idx)

    break

0
apple macbook pro 15
/products/117174/noutbuki/apple-macbook-pro-15-with-retina-display-mid-2015-mjlq2-core-i7-2200-mhz1542880x1800160gb256gbdvd-netintel-iris-pro-graphics-5200wi-fibluetoothmacos-x/
0


In [ ]:
for idx, (name, href) in enumerate(zip(short_names, pc_hrefs)):

    time.sleep(random.randint(1, 5))

    try:
        product_reviews = parse_reviews_for_product(href, random.choice(USER_AGENTS))
        # processing
        if product_reviews:  # if there are less than 10 reviews on the product, nothing is returned

            # reviews
            with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_reviews.txt', 'a', encoding='utf-8') as cf:
                # запись
                cf.write('\n'.join(product_reviews))
                cf.write('\n')

            # name
            with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_names.txt', 'a', encoding='utf-8') as cnf:
                cnf.write(name)
                cnf.write('\n')

            # link
            with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_hrefs.txt', 'a', encoding='utf-8') as chf:
                chf.write(href)
                chf.write('\n')
    except:
        print('There is an error!')

    if idx % 100 == 0:
        print(idx)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_reviews.txt', 'r', encoding='utf-8') as cf:
    data = cf.read().split('\n')

print(data[-1])
print(data[0])
print(len(data[0:-1]))


Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: Трекпад начал пощелкивать спустя 3 месяца после покупки Аллюминиевый корпус вминается, если уронить/ударить ноутбук  Отзыв: Я работаю разработчиком и покупал ноутбук именно для работы. Пользуюсь ноутбуком уже 5-й месяц. До этого пару лет пользовался Thinkpad x220 и Linux. Очень радует то, что операционная система семейства Unix, а также то, что в интернете огромное количество информации под macOS и OS X для разработчиков, когда нужно что-то скомпилировать/собрать и т.д. Это как Linux, только очень качественный и доведенный до ума без его болячек, вроде того, что что-нибудь отвалится в процессе работы само по себе и т.д. Из недостатков: - корпус можно было бы сделать из более прочного материала - я один раз уронил ноутбук на кафель и получил неплохую вмятину на крышке. - не знаю с чем это связано, но трекпад начал немного потрескивать при касании в верхнем правом 

In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_names.txt', 'r', encoding='utf-8') as cf:
    data = cf.read().split('\n')

print(data[-1])
print(data[0])
print(len(data[0:-1]))


apple macbook pro 15
1299
